In [ ]:
import json
import requests
from bs4 import BeautifulSoup

with open("./cookie.json", "r", encoding="UTF-8") as json_file:
    headers = json.load(json_file)

requests_url = "https://www.ceneo.pl/"
product_id = "161123001"
url = f"{requests_url}{product_id}"
response = requests.get(url, headers = headers)

all_opinions = []
while url is not None:
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        page_doc = BeautifulSoup(response.text, 'html.parser')
        opinions = page_doc.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            single_opinion = {}
            single_opinion['opinion_id'] = opinion["data-entry-id"]
            single_opinion['author'] = opinion.select_one("span.user-post__author-name").text.strip()
            try:
                # some opinions do not have a 'recomendation'
                single_opinion['recommendation'] = opinion.select_one("span.user-post__author-recomendation > em").text.strip()
            except AttributeError:
                single_opinion['recommendation'] = None
            single_opinion['score'] = opinion.select_one("span.user-post__score-count").text.strip()
            single_opinion['content'] = opinion.select_one("div.user-post__text").text.strip()
            single_opinion['pros'] = [tag.text.strip() for tag in opinion.select("review-feature__item--positive")] # list comprehension
            single_opinion['cons'] = [tag.text.strip() for tag in opinion.select("review-feature__item--negative")] # list comprehension
            single_opinion['thumbs_up'] = opinion.select_one("button.vote-yes")["data-total-vote"].strip()
            single_opinion['thumbs_down'] = opinion.select_one("button.vote-no")["data-total-vote"].strip()
            single_opinion['date_published'] = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip()
            try:
                single_opinion['date_purchased'] = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"].strip()
            except TypeError:
                single_opinion['date_purchased'] = None
            all_opinions.append(single_opinion)
            print(single_opinion)
        try:
            url = "https://www.ceneo.pl"+page_doc.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None